# Binary Classification Data - File 2 of 2

# Packages

In [1]:
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42) # note that you must use the same seed to ensure consistentcy in your training/validation/testing
tf.random.set_seed(42)

In [2]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications import EfficientNetB7

In [3]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#   tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6144)])

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Data Set Up

In [6]:
data_dir = r"C:\Users\gdlar\Downloads\Files\Binary_Split\Data"
batch_size = 32; #standard batch size, can change as we move along
img_height = 224; #may need to change based on model used (alter at model generation?)
img_width = 224;
n_classes = 3

# Training Dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = batch_size
)

# Validation Dataset
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = batch_size
) 

Found 76391 files belonging to 2 classes.
Using 61113 files for training.
Found 76391 files belonging to 2 classes.
Using 15278 files for validation.


In [6]:
data_dir = r"C:\Users\gdlar\Downloads\Files\Binary_Split\Data"
# batch_size = 32; #standard batch size, can change as we move along
img_height = 224; #may need to change based on model used (alter at model generation?)
img_width = 224;
n_classes = 3

# Training Dataset
train_ds_10 = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size= (img_height, img_width),
    batch_size = 5
)

# Validation Dataset
validation_ds_10 = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (img_height, img_width),
    batch_size = 5
) 

Found 76391 files belonging to 2 classes.
Using 61113 files for training.
Found 76391 files belonging to 2 classes.
Using 15278 files for validation.


# Data Augmentation

In [8]:
# Create an image generator
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
    ]
)
aug_ds = train_ds.map(
  lambda x, y: (data_augmentation(x, training=True), y))

NameError: name 'train_ds' is not defined

# Metrics

In [9]:
metrics = ['AUC', 'accuracy', 'Precision', 'Recall', 'FalseNegatives', 'TrueNegatives']

# Xception

## Model 6

In [9]:
tf.keras.backend.clear_session()

In [10]:
base_model = tf.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)

In [11]:
for layer in base_model.layers:
    layer.trainable = False

model = keras.models.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
# model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [12]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001) # around 

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=metrics)

In [13]:
model.fit(train_ds, 
                  epochs = 5, 
                  steps_per_epoch = 50,
                  validation_data = validation_ds)

Epoch 1/5
50/50 [==============================] - 40s 704ms/step - loss: 391.1012 - auc: 0.5273 - accuracy: 0.5281 - precision: 0.5562 - recall: 0.5529 - false_negatives: 380.0000 - true_negatives: 375.0000 - val_loss: 223.0313 - val_auc: 0.5000 - val_accuracy: 0.4719 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 8069.0000 - val_true_negatives: 7209.0000
Epoch 2/5
50/50 [==============================] - 36s 724ms/step - loss: 78.0917 - auc: 0.5521 - accuracy: 0.5562 - precision: 0.5962 - recall: 0.5812 - false_negatives: 366.0000 - true_negatives: 382.0000 - val_loss: 22.5021 - val_auc: 0.6520 - val_accuracy: 0.6451 - val_precision: 0.6773 - val_recall: 0.6266 - val_false_negatives: 3013.0000 - val_true_negatives: 4800.0000
Epoch 3/5
50/50 [==============================] - 34s 690ms/step - loss: 56.3120 - auc: 0.5551 - accuracy: 0.5581 - precision: 0.5980 - recall: 0.6027 - false_negatives: 350.0000 - true_negatives: 362.0000 - val_loss: 25.5552 - val_a

# ResNet-152V2

In [14]:
tf.keras.backend.clear_session()
from keras.applications.resnet_v2 import ResNet152V2

base_model = ResNet152V2(input_shape=(224, 224,3), include_top=False, weights="imagenet")

for layer in base_model.layers:
    layer.trainable = False

from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
# model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.001), loss = 'binary_crossentropy', metrics = metrics)

model.fit(aug_ds, validation_data = validation_ds, steps_per_epoch = 50, epochs = 5)

Epoch 1/5
50/50 [==============================] - 74s 1s/step - loss: 55259.7734 - auc: 0.5033 - accuracy: 0.4919 - precision: 0.5262 - recall: 0.3799 - false_negatives: 524.0000 - true_negatives: 466.0000 - val_loss: 0.7074 - val_auc: 0.4992 - val_accuracy: 0.5266 - val_precision: 0.5275 - val_recall: 0.9938 - val_false_negatives: 50.0000 - val_true_negatives: 27.0000
Epoch 2/5
50/50 [==============================] - 65s 1s/step - loss: 7.0687 - auc: 0.5110 - accuracy: 0.5469 - precision: 0.5463 - recall: 0.9828 - false_negatives: 15.0000 - true_negatives: 20.0000 - val_loss: 0.7010 - val_auc: 0.4995 - val_accuracy: 0.5274 - val_precision: 0.5279 - val_recall: 0.9960 - val_false_negatives: 32.0000 - val_true_negatives: 21.0000
Epoch 3/5
50/50 [==============================] - 65s 1s/step - loss: 1.0542 - auc: 0.5018 - accuracy: 0.5431 - precision: 0.5441 - recall: 0.9804 - false_negatives: 17.0000 - true_negatives: 17.0000 - val_loss: 0.7007 - val_auc: 0.4996 - val_accuracy: 0.5276

# DenseNet-121

In [14]:
tf.keras.backend.clear_session()
from keras.applications.densenet import DenseNet121
tf.keras.backend.clear_session()
base_model = DenseNet121(input_shape = (224, 224,3), include_top=False,weights='imagenet')

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss='binary_crossentropy',metrics=metrics)

model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)


Epoch 1/5
50/50 [==============================] - 65s 993ms/step - loss: 1.5587 - auc: 0.5838 - accuracy: 0.5531 - precision: 0.5724 - recall: 0.6083 - false_negatives: 331.0000 - true_negatives: 371.0000 - val_loss: 0.6329 - val_auc: 0.6994 - val_accuracy: 0.6444 - val_precision: 0.6823 - val_recall: 0.6112 - val_false_negatives: 3137.0000 - val_true_negatives: 4913.0000
Epoch 2/5
50/50 [==============================] - 45s 906ms/step - loss: 0.7089 - auc: 0.6577 - accuracy: 0.6162 - precision: 0.6342 - recall: 0.6954 - false_negatives: 265.0000 - true_negatives: 381.0000 - val_loss: 0.6130 - val_auc: 0.7182 - val_accuracy: 0.6529 - val_precision: 0.6590 - val_recall: 0.7105 - val_false_negatives: 2336.0000 - val_true_negatives: 4242.0000
Epoch 3/5
50/50 [==============================] - 45s 905ms/step - loss: 0.6397 - auc: 0.7133 - accuracy: 0.6531 - precision: 0.6821 - recall: 0.6766 - false_negatives: 281.0000 - true_negatives: 457.0000 - val_loss: 0.6243 - val_auc: 0.7524 - val

# InceptionResNetV2

In [15]:
tf.keras.backend.clear_session()
# create the base pre-trained model
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = metrics)

# train the model on the new data for a few epochs
model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

Epoch 1/5
50/50 [==============================] - 97s 2s/step - loss: 1.2197 - auc: 0.6060 - accuracy: 0.5863 - precision: 0.6055 - recall: 0.6213 - false_negatives: 320.0000 - true_negatives: 413.0000 - val_loss: 2.8838 - val_auc: 0.6541 - val_accuracy: 0.4775 - val_precision: 0.9143 - val_recall: 0.0119 - val_false_negatives: 7973.0000 - val_true_negatives: 7200.0000
Epoch 2/5
50/50 [==============================] - 72s 1s/step - loss: 0.7031 - auc: 0.6856 - accuracy: 0.6381 - precision: 0.6677 - recall: 0.6970 - false_negatives: 270.0000 - true_negatives: 400.0000 - val_loss: 0.6585 - val_auc: 0.7367 - val_accuracy: 0.6660 - val_precision: 0.7609 - val_recall: 0.5360 - val_false_negatives: 3744.0000 - val_true_negatives: 5850.0000
Epoch 3/5
50/50 [==============================] - 72s 1s/step - loss: 0.6393 - auc: 0.7223 - accuracy: 0.6606 - precision: 0.6978 - recall: 0.6516 - false_negatives: 300.0000 - true_negatives: 496.0000 - val_loss: 0.6964 - val_auc: 0.7619 - val_accuracy

### Model 3

In [16]:
# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = metrics)

# train the model on the new data for a few epochs
model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=7)

Epoch 1/5
50/50 [==============================] - 104s 2s/step - loss: 0.8955 - auc: 0.5838 - accuracy: 0.5606 - precision: 0.5825 - recall: 0.5791 - false_negatives: 354.0000 - true_negatives: 410.0000 - val_loss: 0.6774 - val_auc: 0.7038 - val_accuracy: 0.6521 - val_precision: 0.6737 - val_recall: 0.6618 - val_false_negatives: 2729.0000 - val_true_negatives: 4623.0000
Epoch 2/5
50/50 [==============================] - 74s 1s/step - loss: 0.6864 - auc: 0.6679 - accuracy: 0.6313 - precision: 0.6538 - recall: 0.6956 - false_negatives: 267.0000 - true_negatives: 400.0000 - val_loss: 0.6644 - val_auc: 0.7364 - val_accuracy: 0.6757 - val_precision: 0.6926 - val_recall: 0.6940 - val_false_negatives: 2469.0000 - val_true_negatives: 4724.0000
Epoch 3/5
50/50 [==============================] - 73s 1s/step - loss: 0.6438 - auc: 0.7304 - accuracy: 0.6800 - precision: 0.7099 - recall: 0.7050 - false_negatives: 259.0000 - true_negatives: 469.0000 - val_loss: 0.8442 - val_auc: 0.7620 - val_accurac

### Model 4

In [11]:
# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = metrics)

# train the model on the new data for a few epochs
model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

Epoch 1/5
50/50 [==============================] - 85s 1s/step - loss: 0.8440 - auc: 0.5655 - accuracy: 0.5419 - precision: 0.5661 - recall: 0.5894 - false_negatives: 349.0000 - true_negatives: 366.0000 - val_loss: 0.6420 - val_auc: 0.6881 - val_accuracy: 0.6394 - val_precision: 0.6792 - val_recall: 0.6012 - val_false_negatives: 3218.0000 - val_true_negatives: 4918.0000
Epoch 2/5
50/50 [==============================] - 65s 1s/step - loss: 0.7172 - auc: 0.6305 - accuracy: 0.5969 - precision: 0.6257 - recall: 0.6522 - false_negatives: 304.0000 - true_negatives: 385.0000 - val_loss: 0.6271 - val_auc: 0.7154 - val_accuracy: 0.6524 - val_precision: 0.7273 - val_recall: 0.5469 - val_false_negatives: 3656.0000 - val_true_negatives: 5554.0000
Epoch 3/5
50/50 [==============================] - 66s 1s/step - loss: 0.6613 - auc: 0.6739 - accuracy: 0.6162 - precision: 0.6536 - recall: 0.6447 - false_negatives: 313.0000 - true_negatives: 418.0000 - val_loss: 0.5955 - val_auc: 0.7399 - val_accuracy

## Model 5

In [12]:

# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', metrics = metrics)

# train the model on the new data for a few epochs
model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

Epoch 1/5
50/50 [==============================] - 103s 2s/step - loss: 0.9006 - auc: 0.5490 - accuracy: 0.5381 - precision: 0.5597 - recall: 0.5882 - false_negatives: 348.0000 - true_negatives: 364.0000 - val_loss: 0.6510 - val_auc: 0.6949 - val_accuracy: 0.6423 - val_precision: 0.6897 - val_recall: 0.5867 - val_false_negatives: 3335.0000 - val_true_negatives: 5079.0000
Epoch 2/5
50/50 [==============================] - 78s 2s/step - loss: 0.7194 - auc: 0.6237 - accuracy: 0.5869 - precision: 0.6115 - recall: 0.6586 - false_negatives: 297.0000 - true_negatives: 366.0000 - val_loss: 0.6250 - val_auc: 0.7094 - val_accuracy: 0.6548 - val_precision: 0.6841 - val_recall: 0.6435 - val_false_negatives: 2877.0000 - val_true_negatives: 4812.0000
Epoch 3/5
50/50 [==============================] - 72s 1s/step - loss: 0.6721 - auc: 0.6788 - accuracy: 0.6263 - precision: 0.6474 - recall: 0.6847 - false_negatives: 274.0000 - true_negatives: 407.0000 - val_loss: 0.6419 - val_auc: 0.7544 - val_accurac

## Model 6

In [13]:
# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = InceptionResNetV2(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.00001), loss = 'binary_crossentropy', metrics = metrics)

# train the model on the new data for a few epochs
model.fit(train_ds,
                    steps_per_epoch=50,
                    validation_data=validation_ds,
                    epochs=5)

Epoch 1/5
50/50 [==============================] - 98s 2s/step - loss: 0.8309 - auc: 0.5066 - accuracy: 0.5106 - precision: 0.5324 - recall: 0.6024 - false_negatives: 336.0000 - true_negatives: 308.0000 - val_loss: 0.7041 - val_auc: 0.6064 - val_accuracy: 0.5364 - val_precision: 0.6544 - val_recall: 0.2590 - val_false_negatives: 5979.0000 - val_true_negatives: 6105.0000
Epoch 2/5
50/50 [==============================] - 78s 2s/step - loss: 0.7839 - auc: 0.5112 - accuracy: 0.5200 - precision: 0.5631 - recall: 0.6162 - false_negatives: 342.0000 - true_negatives: 283.0000 - val_loss: 0.6741 - val_auc: 0.6297 - val_accuracy: 0.5787 - val_precision: 0.6466 - val_recall: 0.4464 - val_false_negatives: 4467.0000 - val_true_negatives: 5240.0000
Epoch 3/5
50/50 [==============================] - 82s 2s/step - loss: 0.7393 - auc: 0.5408 - accuracy: 0.5269 - precision: 0.5538 - recall: 0.6214 - false_negatives: 326.0000 - true_negatives: 308.0000 - val_loss: 0.6705 - val_auc: 0.6577 - val_accuracy

# Efficent B7

## Model 1

In [11]:
# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = EfficientNetB7(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy', 
              metrics = ['AUC', 'accuracy', 'Precision', 'Recall', 'FalseNegatives', 'TrueNegatives'])

# train the model on the new data for a few epochs
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0]']      

In [12]:
model.fit(train_ds_10,
                    steps_per_epoch=50,
                    validation_data=validation_ds_10,
                    epochs=5,
                    batch_size = 10)

Epoch 1/5
50/50 [==============================] - 293s 5s/step - loss: 0.9081 - auc: 0.6513 - accuracy: 0.6040 - precision: 0.6220 - recall: 0.6077 - false_negatives: 51.0000 - true_negatives: 72.0000 - val_loss: 0.7006 - val_auc: 0.6353 - val_accuracy: 0.6020 - val_precision: 0.6157 - val_recall: 0.6553 - val_false_negatives: 2781.0000 - val_true_negatives: 3909.0000
Epoch 2/5
50/50 [==============================] - 244s 5s/step - loss: 0.9672 - auc: 0.6412 - accuracy: 0.6200 - precision: 0.6281 - recall: 0.6032 - false_negatives: 50.0000 - true_negatives: 79.0000 - val_loss: 0.7505 - val_auc: 0.6535 - val_accuracy: 0.5909 - val_precision: 0.5763 - val_recall: 0.8509 - val_false_negatives: 1203.0000 - val_true_negatives: 2162.0000
Epoch 3/5
50/50 [==============================] - 243s 5s/step - loss: 0.8016 - auc: 0.6990 - accuracy: 0.6560 - precision: 0.7039 - recall: 0.7230 - false_negatives: 41.0000 - true_negatives: 57.0000 - val_loss: 0.7798 - val_auc: 0.6788 - val_accuracy: 0

## Model 2

In [14]:
# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = EfficientNetB7(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'binary_crossentropy', 
              metrics = ['AUC', 'accuracy', 'Precision', 'Recall', 'FalseNegatives', 'TrueNegatives'])



In [16]:
# train the model on the new data for a few epochs
model.fit(train_ds_10,
                    steps_per_epoch=50,
                    validation_data=validation_ds_10,
                    epochs=5)

Epoch 1/5
50/50 [==============================] - 260s 5s/step - loss: 1.0025 - auc: 0.5556 - accuracy: 0.5280 - precision: 0.5528 - recall: 0.5191 - false_negatives: 63.0000 - true_negatives: 64.0000 - val_loss: 0.7032 - val_auc: 0.6021 - val_accuracy: 0.5682 - val_precision: 0.6318 - val_recall: 0.4374 - val_false_negatives: 4540.0000 - val_true_negatives: 5152.0000
Epoch 2/5
50/50 [==============================] - 241s 5s/step - loss: 0.7576 - auc: 0.6212 - accuracy: 0.5880 - precision: 0.6032 - recall: 0.5891 - false_negatives: 53.0000 - true_negatives: 71.0000 - val_loss: 0.6918 - val_auc: 0.6507 - val_accuracy: 0.5998 - val_precision: 0.6945 - val_recall: 0.4324 - val_false_negatives: 4580.0000 - val_true_negatives: 5674.0000
Epoch 3/5
50/50 [==============================] - 239s 5s/step - loss: 0.7905 - auc: 0.5839 - accuracy: 0.5600 - precision: 0.5778 - recall: 0.7536 - false_negatives: 34.0000 - true_negatives: 36.0000 - val_loss: 0.6675 - val_auc: 0.6405 - val_accuracy: 0

## Model 3

In [19]:
# create the base pre-trained model
tf.keras.backend.clear_session()
base_model = EfficientNetB7(input_shape = (224, 224,3), include_top = False, weights = 'imagenet')
x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.7)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = 'binary_crossentropy', 
              metrics = ['AUC', 'accuracy', 'Precision', 'Recall', 'FalseNegatives', 'TrueNegatives'])

# train the model on the new data for a few epochs


In [21]:
model.fit(train_ds_10,
                    steps_per_epoch=50,
                    validation_data=validation_ds_10,
                    epochs=5)

Epoch 1/5
50/50 [==============================] - 271s 5s/step - loss: 12.5002 - auc: 0.4943 - accuracy: 0.4720 - precision: 0.4622 - recall: 0.4472 - false_negatives: 68.0000 - true_negatives: 63.0000 - val_loss: 45.3502 - val_auc: 0.5443 - val_accuracy: 0.4800 - val_precision: 0.6771 - val_recall: 0.0296 - val_false_negatives: 7830.0000 - val_true_negatives: 7095.0000
Epoch 2/5
50/50 [==============================] - 241s 5s/step - loss: 1.4156 - auc: 0.4434 - accuracy: 0.5080 - precision: 0.5275 - recall: 0.8519 - false_negatives: 20.0000 - true_negatives: 12.0000 - val_loss: 0.9865 - val_auc: 0.5181 - val_accuracy: 0.5110 - val_precision: 0.5481 - val_recall: 0.4224 - val_false_negatives: 4661.0000 - val_true_negatives: 4399.0000
Epoch 3/5
50/50 [==============================] - 251s 5s/step - loss: 0.7120 - auc: 0.4632 - accuracy: 0.5560 - precision: 0.5605 - recall: 0.9858 - false_negatives: 2.0000 - true_negatives: 0.0000e+00 - val_loss: 0.7360 - val_auc: 0.5439 - val_accurac